In [9]:
push!(LOAD_PATH, "/home/zhenan/Github/AtomicOpt.jl")
using AtomicOpt
using LinearAlgebra
using SparseArrays
using Printf
using Arpack

## Load data

In [10]:
function load_data(data::String)
    if data == "temperature"
        file = open("./temp2020.csv", "r")
        I = Vector{Int64}(); J = Vector{Int64}(); V = Vector{Float64}()
        numRows=0; RowDict = Dict{String, Int64}();
        numCols=0; ColDict = Dict{String, Int64}();
        while !eof(file)
            line = readline(file)
            info = split(line, ",")
            # get value
            v = parse(Float64, info[3])
            if v!= 0
                push!(V, v)
                # get row index
                if !haskey(RowDict, info[1])
                    numRows += 1
                    RowDict[info[1]] = numRows
                end
                i = RowDict[info[1]]
                push!(I, i)
                # get column index
                if !haskey(ColDict, info[2])
                    numCols += 1
                    ColDict[info[2]] = numCols
                end
                j = ColDict[info[2]]
                push!(J, j)
            end    
        end
        close(file)
        X = sparse(I,J,V,numRows,numCols)
        X ./= maximum(X)
        # low rank approximation
        m, n = size(X)
        r = 5
        U, S, V = svds(X, nsv=r)[1]
        Xopt = U*Diagonal(S)*V'
        τ = sum(S)
        # mask 
        p = 0.5
        mask = sprand(Bool, m, n, p)
        mask = convert(SparseMatrixCSC{Float64, Int64}, mask)
        I,J,V = findnz(mask)
        # measurement
        b = Vector{Float64}()
        bopt = Vector{Float64}()
        l = length(I)
        for t in 1:l
            i, j = I[t], J[t]
            push!(b, X[i,j])
            push!(bopt, Xopt[i,j])
        end
        α = norm(b - bopt)^2/2
        return m, n, r, Xopt, mask, b, α, τ
    elseif data == "movie"
        file = open("examples/MatrixCompletion/ml-100k.txt", "r")
        I = Vector{Int64}(); J = Vector{Int64}(); V = Vector{Float64}()
        while !eof(file)
            line = readline(file)
            info = split(line)
            i = parse(Int64, info[1])
            j = parse(Int64, info[2])
            v = parse(Float64, info[3])
            push!(I, i); push!(J, j); push!(V, v)    
        end
        close(file)
        X = sparse(I,J,V,maximum(I),maximum(J))
        X ./= maximum(X)
        # low rank approximation
        m, n = size(X)
        r = 5
        U, S, V = svds(X, nsv=r)[1]
        Xopt = U*Diagonal(S)*V'
        τ = sum(S)
        # mask 
        p = 0.5
        mask = sprand(Bool, m, n, p)
        mask = convert(SparseMatrixCSC{Float64, Int64}, mask)
        I,J,V = findnz(mask)
        # measurement
        b = Vector{Float64}()
        bopt = Vector{Float64}()
        l = length(I)
        for t in 1:l
            i, j = I[t], J[t]
            push!(b, X[i,j])
            push!(bopt, Xopt[i,j])
        end
        α = norm(b - bopt)^2/2
        return m, n, r, Xopt, mask, b, α, τ
    elseif data == "random"
        m, n, r = 100, 100, 3
        X = rand(m, n)
        U, S, V = svd(X)
        X = U[:,1:r] * Diagonal( S[1:r] ) * V[:,1:r]'
        τ = sum(S[1:r])
        # mask 
        p = 0.5
        mask = sprand(Bool, m, n, p)
        mask = convert(SparseMatrixCSC{Float64, Int64}, mask)
        I,J,V = findnz(mask)
        # measurement 
        b = Vector{Float64}()
        l = length(I)
        for t in 1:l
            i, j = I[t], J[t]
            push!(b, X[i,j])
        end
        η = rand(l)
        ϵ = 1; η .*= (ϵ/norm(η))
        b .+= η
        α = norm(η)^2/2
        return m, n, r, X, mask, b, α, τ
    else
        println("invalid data")
        return
    end
end

load_data (generic function with 1 method)

In [11]:
m, n, r, Xopt, mask, b, αopt, τopt = load_data("temperature");

## Solve matrix completion problem

In [ ]:
Mop = MaskOP(mask)
A = NucBall(m, n, 2*r)
sol = level_set(Mop, b, A, α = αopt, tol = 1e-3, maxIts=length(b))


  -------------------------------------------------------------------------
  Polar Level Set Method
  -------------------------------------------------------------------------
  number of variables    2488068         number of constraints 1244708
  feasibility tolerance  2.81e-01         α                    2.38e+03
  max iterations         1244708 
  -------------------------------------------------------------------------
  Major      Minor        u-α        ℓ-α        gap          τ         infeas-α  Subproblem
      1          2   1.15e+04   7.03e+03   4.42e+03   2.15e+02       3.67e+04   suboptimal
      2          2   5.62e+03   5.44e+03   1.87e+02   3.12e+02       3.67e+04   suboptimal
      3          2   1.83e+03   1.12e+03   7.06e+02   4.22e+02       3.67e+04   suboptimal
      4          4   9.60e+02   1.40e+02   8.20e+02   4.72e+02       3.67e+04   suboptimal
      5          3   8.34e+02   4.09e+02   4.25e+02   4.81e+02       3.67e+04   suboptimal
      6          6   5

## Report relative difference

In [7]:
x = constructPrimal(sol)
X = reshape(x, m, n)
@printf "The relative difference between Xopt and X: %e \n" norm(Xopt - X)/norm(Xopt)

The relative difference between Xopt and X: 2.609357e-02 
